### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [2]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [3]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    fullset=reviews.sort_values(by=order_by).head(10000)
    training_df=fullset.head(training_size)
    validation_df=fullset.tail(testing_size)
    return training_df, validation_df

In [4]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [5]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [6]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [7]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 10.526492
2 		 5.931230
3 		 4.162767
4 		 3.128239
5 		 2.447297
6 		 1.965820
7 		 1.609026
8 		 1.335946
9 		 1.121890
10 		 0.950902
11 		 0.812152
12 		 0.698042
13 		 0.603119
14 		 0.523397
15 		 0.455914
16 		 0.398430
17 		 0.349216
18 		 0.306909
19 		 0.270411
20 		 0.238827
21 		 0.211415
22 		 0.187558
23 		 0.166742
24 		 0.148532
25 		 0.132566
26 		 0.118534
27 		 0.106178
28 		 0.095275
29 		 0.085637
30 		 0.077101
31 		 0.069529
32 		 0.062800
33 		 0.056810
34 		 0.051470
35 		 0.046700
36 		 0.042434
37 		 0.038610
38 		 0.035179
39 		 0.032094
40 		 0.029317
41 		 0.026812
42 		 0.024550
43 		 0.022504
44 		 0.020650
45 		 0.018969
46 		 0.017442
47 		 0.016053
48 		 0.014788
49 		 0.013634
50 		 0.012581
51 		 0.011619
52 		 0.010738
53 		 0.009932
54 		 0.009193
55 		 0.008514
56 		 0.007891
57 		 0.007319
58 		 0.006792
59 		 0.006306
60 		 0.005859
61 		 0.005447
62 		 0.005066
63 		 0.004715
64 		 

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [8]:
train_data_df.head(5)

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
np.where(train_data_df.index==66)[0][0]

4

In [10]:
np.where(train_data_df.columns==8133)[0][0]

1

In [11]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Use the training data to create a series of users and movies that matches the ordering in training data
    
    # User row and Movie Column
    row=np.where(train_data_df.index==user_id)[0][0]
    column=np.where(train_data_df.columns==movie_id)[0][0]
    # Take dot product of that row and column in U and V to make prediction
    pred=np.dot(user_matrix[row,:],movie_matrix[:,column])
    return pred

In [12]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

5.2574637387213325

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [13]:
reviews.query("user_id==8")

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
35,8,385002,10,1363061522,2013-03-12 04:12:02,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
36,8,1220198,10,1367222049,2013-04-29 07:54:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
37,8,1462900,10,1363946231,2013-03-22 09:57:11,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
38,8,1512685,10,1364661242,2013-03-30 16:34:02,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
39,8,1631707,10,1367919290,2013-05-07 09:34:50,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
40,8,1986994,1,1364221582,2013-03-25 14:26:22,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
41,8,1999995,10,1367919361,2013-05-07 09:36:01,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [14]:
movies.query("movie_id==2844")[['movie','genre']].values[0][0]

"Fantômas - À l'ombre de la guillotine (1913)"

In [15]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    name=movies.query("movie_id==@movie_id")[['movie','genre']].values[0][0]
    genre=movies.query("movie_id==@movie_id")[['movie','genre']].values[0][1]
    try:
        rate=reviews.query("user_id==@user_id and movie_id==@movie_id")[['rating','date']].values[0][0]
        date=reviews.query("user_id==@user_id and movie_id==@movie_id")[['rating','date']].values[0][1]
        print("user {} rated movie '{}'({}) {}/10 on {}. The prediction is {}".format(user_id, name,genre,rate, date,prediction))
    except IndexError:
        print("user {} haven't watch movie '{}'({}). The prediction is {}".format(user_id, name,genre,prediction))

In [16]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

user 8 haven't watch movie 'Fantômas - À l'ombre de la guillotine (1913)'(Crime|Drama). The prediction is 5.2574637387213325


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well we have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [33]:
val_df.head(7)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
650588,49056,1598822,8,1363308721,2013-03-15 00:52:01,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650569,49056,289879,9,1363308742,2013-03-15 00:52:22,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650585,49056,1563738,9,1363308780,2013-03-15 00:53:00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650583,49056,1458175,4,1363308799,2013-03-15 00:53:19,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
378686,28599,103639,8,1363309112,2013-03-15 00:58:32,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
667660,50593,1560985,4,1363309202,2013-03-15 01:00:02,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
385306,29000,287978,9,1363309214,2013-03-15 01:00:14,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [32]:
val_df['rating'].values

array([8, 9, 9, ..., 8, 7, 6])

In [38]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    user_id=val_df['user_id'].values
    movie_id=val_df['movie_id'].values
    raing=val_df['rating'].values
    for i in range(num_preds):
        pre=predict_rating(user_mat, movie_mat, user_id[i], movie_id[i])
        print("user {} rated movie {} {}/10, prediction is {}".format(user_id[i],movie_id[i], raing[i], pre))
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)        

user 49056 rated movie 1598822 8/10, prediction is 8.112940237414593
user 49056 rated movie 289879 9/10, prediction is 8.044858282939451
user 49056 rated movie 1563738 9/10, prediction is 6.547716436087137
user 49056 rated movie 1458175 4/10, prediction is 7.618978851799706
user 28599 rated movie 103639 8/10, prediction is 7.7434881830087
user 50593 rated movie 1560985 4/10, prediction is 4.302086314589652


In [56]:
[i for i in range(6)]

[0, 1, 2, 3, 4, 5]

In [39]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

user 49056 rated movie 1598822 8/10, prediction is 8.112940237414593
user 49056 rated movie 289879 9/10, prediction is 8.044858282939451
user 49056 rated movie 1563738 9/10, prediction is 6.547716436087137
user 49056 rated movie 1458175 4/10, prediction is 7.618978851799706
user 28599 rated movie 103639 8/10, prediction is 7.7434881830087
user 50593 rated movie 1560985 4/10, prediction is 4.302086314589652


IndexError: index 0 is out of bounds for axis 0 with size 0

In [58]:
val_df.iloc[6:7,:]

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
385306,29000,287978,9,1363309214,2013-03-15 01:00:14,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [65]:
np.where(train_data_df.columns==287978)

(array([], dtype=int64),)

Train set doesn't have moive with id 287978